In [1]:
import os
import pickle
import random
import math
import time

import mxnet as mx
import numpy as np
import pandas as pd
import cv2

from mxnet import autograd, gluon, init
from mxnet import ndarray as nd
from mxnet.gluon.data import DataLoader, dataset
from mxnet.gluon import nn, loss as gloss, data as gdata

from sklearn import neighbors
from sklearn import preprocessing
from scipy.spatial.distance import pdist, cdist
from numpy import linalg

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
x_0 = pickle.load(open('output/train/x_0.pkl', 'rb'))

In [3]:
counts, deletes = [], []
for i in range(len(x_0[0])):
    counts.append(x_0[0][i].sum())

counts = pd.Series(counts)
counts.value_counts().sort_index().plot()

# Baseline-NearestNeighbors

In [4]:
x = np.array(x_0[0]).astype(np.float32)
y = np.array(x_0[1]).astype(np.float32)

In [5]:
test = pickle.load(open('output/test_500_2.pkl', 'rb'))
x_test = np.array(test[0]).astype(np.float32)
y_test = np.array(test[1]).astype(np.float32)

In [27]:
X = []
for i in range(len(x)):
    X.append(x[i].flatten())

X_test = []
for i in range(len(x_test)):
    X_test.append(x_test[i].flatten())

X = np.array(X).astype(np.float32)
X_test = np.array(X_test).astype(np.float32)

In [28]:
print(X.shape,y.shape,X_test.shape,y_test.shape)

(6566, 24000) (6566,) (500, 24000) (500,)


In [29]:
start_time = time.clock()
# Find Nearest Neighbors for test in train base
mrr, acc1, acc3, acc5, acc10, length = 0.0, 0.0, 0.0, 0.0, 0.0, len(x_test)

nbrs = neighbors.NearestNeighbors(n_neighbors=10, algorithm='ball_tree').fit(X)
nbrs_all = neighbors.NearestNeighbors(n_neighbors=len(x), algorithm='ball_tree').fit(X)

end_time = time.clock()
print('Training time: %s Seconds'%(end_time-start_time))

start_time = time.clock()
distances, indices = nbrs.kneighbors(X_test)
distances_all, indices_all = nbrs_all.kneighbors(X_test)
    
for i in range(len(indices)):
        
    y_true = y_test[i] # True label for test        
    y_predict = y[indices[i]] # Predict for test
    y_predict_all = y[indices_all[i]]
        
    rank = np.argwhere(y_predict_all == y_true)[0][0] + 1
    mrr += 1/rank
        
    #print(y_true, y_predict, indices[i])
        
    if y_true in y_predict[:1]:
        acc1 += 1
        
    if y_true in y_predict[:3]:
        acc3 += 1
        
    if y_true in y_predict[:5]:
        acc5 += 1
        
    if y_true in y_predict:
        acc10 += 1
        
    #print(i,acc1,acc3,acc5,acc10)
        
acc1 /= length
acc3 /= length
acc5 /= length
acc10 /= length
mrr /= length

end_time = time.clock()

print(acc1,acc3,acc5,acc10,mrr)
print('Predicting time: %s Seconds'%(end_time-start_time))

Training time: 13.352514999999812 Seconds
0.432 0.488 0.504 0.528 0.46639909505212856
Predicting time: 189.21340299999997 Seconds


In [31]:
start_time = time.clock()
# Find Nearest Neighbors for test in train base
mrr, acc1, acc3, acc5, acc10, length = 0.0, 0.0, 0.0, 0.0, 0.0, len(x_test)

nbrs = neighbors.NearestNeighbors(n_neighbors=10, algorithm='kd_tree').fit(X)
nbrs_all = neighbors.NearestNeighbors(n_neighbors=len(x), algorithm='kd_tree').fit(X)

end_time = time.clock()
print('Training time: %s Seconds'%(end_time-start_time))

start_time = time.clock()
distances, indices = nbrs.kneighbors(X_test)
distances_all, indices_all = nbrs_all.kneighbors(X_test)
    
for i in range(len(indices)):
        
    y_true = y_test[i] # True label for test        
    y_predict = y[indices[i]] # Predict for test
    y_predict_all = y[indices_all[i]]
        
    rank = np.argwhere(y_predict_all == y_true)[0][0] + 1
    mrr += 1/rank
        
    #print(y_true, y_predict, indices[i])
        
    if y_true in y_predict[:1]:
        acc1 += 1
        
    if y_true in y_predict[:3]:
        acc3 += 1
        
    if y_true in y_predict[:5]:
        acc5 += 1
        
    if y_true in y_predict:
        acc10 += 1
        
    #print(i,acc1,acc3,acc5,acc10)
        
acc1 /= length
acc3 /= length
acc5 /= length
acc10 /= length
mrr /= length

end_time = time.clock()

print(acc1,acc3,acc5,acc10,mrr)
print('Predicting time: %s Seconds'%(end_time-start_time))

Training time: 11.842715999999882 Seconds
0.43 0.488 0.504 0.528 0.46634770479242765
Predicting time: 245.98267099999998 Seconds


In [32]:
start_time = time.clock()
# Find Nearest Neighbors for test in train base
mrr, acc1, acc3, acc5, acc10, length = 0.0, 0.0, 0.0, 0.0, 0.0, len(x_test)

nbrs = neighbors.NearestNeighbors(n_neighbors=10, algorithm='brute').fit(X)
nbrs_all = neighbors.NearestNeighbors(n_neighbors=len(x), algorithm='brute').fit(X)

end_time = time.clock()
print('Training time: %s Seconds'%(end_time-start_time))

start_time = time.clock()
distances, indices = nbrs.kneighbors(X_test)
distances_all, indices_all = nbrs_all.kneighbors(X_test)
    
for i in range(len(indices)):
        
    y_true = y_test[i] # True label for test        
    y_predict = y[indices[i]] # Predict for test
    y_predict_all = y[indices_all[i]]
        
    rank = np.argwhere(y_predict_all == y_true)[0][0] + 1
    mrr += 1/rank
        
    #print(y_true, y_predict, indices[i])
        
    if y_true in y_predict[:1]:
        acc1 += 1
        
    if y_true in y_predict[:3]:
        acc3 += 1
        
    if y_true in y_predict[:5]:
        acc5 += 1
        
    if y_true in y_predict:
        acc10 += 1
        
    #print(i,acc1,acc3,acc5,acc10)
        
acc1 /= length
acc3 /= length
acc5 /= length
acc10 /= length
mrr /= length

end_time = time.clock()

print(acc1,acc3,acc5,acc10,mrr)
print('Predicting time: %s Seconds'%(end_time-start_time))

Training time: 0.20479199999999764 Seconds
0.43 0.488 0.504 0.526 0.4654186546118588
Predicting time: 9.501111999999921 Seconds


## Data Augment

In [4]:
RANGE = len(x_0[0])
SAMPLENUM = 20
BASE = 10

In [5]:
indices = np.load('output/train/indices.npy')

In [6]:
x_1 = []
for i in range(1,SAMPLENUM+1):
    print(i)
    x_1.append(pickle.load(open('output/train/x_'+str(i)+'.pkl', 'rb')))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


In [7]:
X1, X2, X3, Y = [], [], [], []
for i in range(RANGE):
    for j in range(SAMPLENUM):
        X1.append(x_0[0][i])
        X2.append(x_1[j][i])
        X3.append(x_1[np.random.randint(0, SAMPLENUM)][int(indices[i][BASE+j])])
        Y.append(x_0[1][i])

In [ ]:
X1 = np.array(X1).astype(np.float32)
X2 = np.array(X2).astype(np.float32)
X3 = np.array(X3).astype(np.float32)    
Y = np.array(Y).astype(np.float32)

In [ ]:
x = np.array(x_0[0]).astype(np.float32)
y = np.array(x_0[1]).astype(np.float32)

In [ ]:
test = [[], []]
for i in range(500):
    a = np.random.randint(0, SAMPLENUM)
    b = np.random.randint(0, RANGE)
    test[0].append(x_1[a][b])
    test[1].append(x_0[1][b])

In [10]:
test = pickle.load(open('output/test_500_2.pkl', 'rb'))
x_test = np.array(test[0]).astype(np.float32)
y_test = np.array(test[1]).astype(np.float32)

## Model

In [11]:
def takeFirst(elem):
    return elem[0]

In [12]:
def conv_block(num_channels):
    blk = nn.Sequential()
    blk.add(nn.BatchNorm(), nn.Activation('relu'),
            nn.Conv2D(num_channels, kernel_size=3, padding=1))
    return blk

class DenseBlock(nn.Block):
    def __init__(self, num_convs, num_channels, **kwargs):
        super(DenseBlock, self).__init__(**kwargs)
        self.net = nn.Sequential()
        for _ in range(num_convs):
            self.net.add(conv_block(num_channels))

    def forward(self, X):
        for blk in self.net:
            Y = blk(X)
            X = nd.concat(X, Y, dim=1)  # 在通道维上将输入和输出连结
        return X

def transition_block(num_channels):
    blk = nn.Sequential()
    blk.add(nn.BatchNorm(), nn.Activation('relu'),
            nn.Conv2D(num_channels, kernel_size=1),
            nn.AvgPool2D(pool_size=2, strides=2))
    return blk

def DenseNet():
    net = nn.Sequential()
    net.add(nn.Conv2D(64, kernel_size=7, strides=2, padding=3),
            nn.BatchNorm(), nn.Activation('relu'),
            nn.MaxPool2D(pool_size=2, strides=1, padding=1))
    
    num_channels, growth_rate = 64, 32  # num_channels为当前的通道数
    num_convs_in_dense_blocks = [4, 4, 4, 4]

    for i, num_convs in enumerate(num_convs_in_dense_blocks):
        net.add(DenseBlock(num_convs, growth_rate))
        # 上一个稠密块的输出通道数
        num_channels += num_convs * growth_rate
        # 在稠密块之间加入通道数减半的过渡层
        if i != len(num_convs_in_dense_blocks) - 1:
            net.add(transition_block(num_channels // 2))
    
    net.add(nn.BatchNorm(), nn.Activation('relu'), nn.GlobalAvgPool2D(), nn.Dense(128))
    return net

In [13]:
def spatial_pyramid_pool(previous_conv, num_sample, shape, out_pool_size):
    '''
    previous_conv: a tensor vector of previous convolution layer
    num_sample: an int number of image in the batch
    previous_conv_size: an int vector [height, width] of the matrix features size of previous convolution layer
    out_pool_size: a int vector of expected output size of max pooling layer
    
    returns: a tensor vector with shape [1 x n] is the concentration of multi-level pooling
    '''
    ## NCHW
    N = shape[0]
    C = shape[1]
    H = shape[2]
    W = shape[3]
    
    for i in range(len(out_pool_size)):
        h_wid = int(math.ceil(H / out_pool_size[i]))
        w_wid = int(math.ceil(W / out_pool_size[i]))
        
        h_pad = int((h_wid*out_pool_size[i] - H + 1) / 2)
        w_pad = int((w_wid*out_pool_size[i] - W + 1) / 2)
        
        maxpool = nn.MaxPool2D(pool_size = (h_wid, w_wid), strides=(h_wid, w_wid), padding=(h_pad, w_pad))
        x = maxpool(previous_conv)
        #print(x.shape)
        
        if(i == 0):
            spp = x.reshape((N, -1))
        else:
            spp = nd.concat(
                spp, 
                x.reshape((N, -1)), 
                dim=1
            )
    
    return spp

In [36]:
def alexNet():
    net = nn.Sequential()
    # 使用较大的11 x 11窗口来捕获物体。同时使用步幅4来较大幅度减小输出高和宽。这里使用的输出通
    # 道数比LeNet中的也要大很多
    net.add(nn.Conv2D(channels=96, kernel_size=11, strides=4,activation='relu'),
            #nn.MaxPool2D(pool_size=4, strides=2),
            # 减小卷积窗口，使用填充为2来使得输入与输出的高和宽一致，且增大输出通道数
            nn.Conv2D(256, kernel_size=5, padding=2, activation='relu'),
            #nn.MaxPool2D(pool_size=4, strides=2),
            # 连续3个卷积层，且使用更小的卷积窗口。除了最后的卷积层外，进一步增大了输出通道数。
            # 前两个卷积层后不使用池化层来减小输入的高和宽
            nn.Conv2D(384, kernel_size=5, padding=1, activation='relu'),
            nn.Conv2D(384, kernel_size=3, padding=1, activation='relu'),
            nn.Conv2D(256, kernel_size=3, padding=1, activation='relu'),
            nn.MaxPool2D(pool_size=3, strides=2),
            # 这里全连接层的输出个数比LeNet中的大数倍。使用丢弃层来缓解过拟合
            nn.Dense(4096, activation="relu"), nn.Dropout(0.5),
            nn.Dense(4096, activation="relu"), nn.Dropout(0.5),
            nn.Dense(128))
    return net

In [15]:
def vgg_block(num_convs, num_channels):
    blk = nn.Sequential()
    for _ in range(num_convs):
        blk.add(nn.Conv2D(num_channels, kernel_size=3, padding=1, activation='relu'))
    blk.add(nn.MaxPool2D(pool_size=2, strides=2, padding=1))
    return blk

def vgg(conv_arch):
    net = nn.Sequential()
    # 卷积层部分
    for (num_convs, num_channels) in conv_arch:
        net.add(vgg_block(num_convs, num_channels))
    # 全连接层部分
    net.add(nn.Dense(4096, activation='relu'), nn.Dropout(0.5),
            nn.Dense(4096, activation='relu'), nn.Dropout(0.5),
            nn.Dense(128))
    return net

In [16]:
def baseNet():
    base_net = nn.Sequential()
    with base_net.name_scope():
        base_net.add(nn.Dense(256, activation='relu'))
        base_net.add(nn.Dense(128, activation='relu'))
    return base_net

In [17]:
class ArrayDataset():
    """A dataset that combines multiple dataset-like objects, e.g.
    Datasets, lists, arrays, etc.

    The i-th sample is defined as `(x1[i], x2[i], ...)`.

    Parameters
    ----------
    *args : one or more dataset-like objects
        The data arrays.
    """
    def __init__(self, *args):
        assert len(args) > 0, "Needs at least 1 arrays"
        self._length = len(args[0])
        self._data = []
        for i, data in enumerate(args):
            assert len(data) == self._length, \
                "All arrays must have the same length; array[0] has length %d " \
                "while array[%d] has %d." % (self._length, i+1, len(data))
            if isinstance(data, nd.NDArray) and len(data.shape) == 1:
                data = data.asnumpy()
            self._data.append(data)

    def __getitem__(self, idx):
        if len(Xself._data) == 1:
            return self._data[0][idx]
        else:
            return tuple(data[idx] for data in self._data)

    def __len__(self):
        return self._length

In [18]:
def evaluate_net(net, train_iter, test_iter, ctx):
    
    # Predict for train
    for i, (data, label) in enumerate(train_iter):
        data = data.as_in_context(ctx)
        label = label.as_in_context(ctx)
        
        if i == 0:
            X = net(data).asnumpy()
            Y = label.asnumpy()
        else:
            X = np.concatenate((X, net(data).asnumpy()))
            Y = np.concatenate((Y, label.asnumpy()))

    #normalizer = preprocessing.Normalizer().fit(X)
    #X = normalizer.transform(X)
    nbrs = neighbors.NearestNeighbors(n_neighbors=10, algorithm='ball_tree').fit(X)
    nbrs_all = neighbors.NearestNeighbors(n_neighbors=len(X), algorithm='ball_tree').fit(X)
    
    # Predict for test
    for i, (test, label) in enumerate(test_iter):
        test = test.as_in_context(ctx)
        label = label.as_in_context(ctx)
        
        if i == 0:
            X_test = net(test).asnumpy()
            Y_test = label.asnumpy()
        else:
            X_test = np.concatenate((X_test, net(test).asnumpy()))
            Y_test = np.concatenate((Y_test, label.asnumpy()))
            
    start_time = time.clock()
    
    # Find Nearest Neighbors for test in train base
    mrr, acc1, acc3, acc5, acc10, length = 0.0, 0.0, 0.0, 0.0, 0.0, len(X_test)
    #X_test = normalizer.transform(X_test)    
    distances, indices = nbrs.kneighbors(X_test)
    distances_all, indices_all = nbrs_all.kneighbors(X_test)
    
    for i in range(len(indices)):
        
        y_true = Y_test[i] # True label for test        
        y_predict = Y[indices[i]] # Predict for test
        y_predict_all = Y[indices_all[i]]
        
        rank = np.argwhere(y_predict_all == y_true)[0][0] + 1
        mrr += 1/rank
        
        #print(y_true, y_predict, indices[i])
        
        if y_true in y_predict[:1]:
            acc1 += 1
        
        if y_true in y_predict[:3]:
            acc3 += 1
        
        if y_true in y_predict[:5]:
            acc5 += 1
        
        if y_true in y_predict:
            acc10 += 1
    
    end_time = time.clock()
        
    acc1 /= length
    acc3 /= length
    acc5 /= length
    acc10 /= length
    mrr /= length
    
    predict_time = end_time - start_time
    
    return [mrr,acc1,acc3,acc5,acc10,predict_time]

In [19]:
# coding: utf-8
def save(params, filename, strip_prefix=''):
    """Save parameters to file.

    Parameters
    ----------
    filename : str
        Path to parameter file.
    strip_prefix : str, default ''
        Strip prefix from parameter names before saving.
    """
    arg_dict = {}
    for param in params.values():
        weight = param._reduce()
        if not param.name.startswith(strip_prefix):
            raise ValueError(
                "Prefix '%s' is to be striped before saving, but Parameter's "
                "name '%s' does not start with '%s'. "
                "this may be due to your Block shares parameters from other "
                "Blocks or you forgot to use 'with name_scope()' when creating "
                "child blocks. For more info on naming, please see "
                "http://mxnet.incubator.apache.org/tutorials/basic/naming.html"%(
                    strip_prefix, param.name, strip_prefix))
        arg_dict[param.name[len(strip_prefix):]] = weight
    
    #print(arg_dict)   
    mx.nd.save(filename, arg_dict)

In [44]:
class SPP_CNN(nn.Block):
    '''
    A CNN model which adds spp layer so that we can input multi-size tensor
    '''
    def __init__(self, d=218, **kwargs):
        super(SPP_CNN, self).__init__(**kwargs)
        with self.name_scope():
            self.output_num = [4, 2, 1]
            self.d = d

            self.conv1 = nn.Conv2D(channels=96, kernel_size=11, strides=4,activation='relu')

            self.conv2 = nn.Conv2D(channels=256, kernel_size=7, padding=2, activation='relu')
            self.BN1 = nn.BatchNorm()

            self.conv3 = nn.Conv2D(channels=384, kernel_size=5, padding=1, activation='relu')
            self.BN2 = nn.BatchNorm()

            #self.conv4 = nn.Conv2D(channels=384, kernel_size=3, padding=1, activation='relu')
            #self.BN3 = nn.BatchNorm()

            #self.conv5 = nn.Conv2D(channels=256, kernel_size=3, padding=1, activation='relu')
            #self.BN4 = nn.BatchNorm()

#             self.pool1 = nn.MaxPool2D(pool_size=3, strides=2)
    #         self.pool2 = nn.MaxPool2D(pool_size=3, strides=2)
    #         self.pool3 = nn.MaxPool2D(pool_size=3, strides=2)

            self.drop = nn.Dropout(0.3)

            self.fc1 = nn.Dense(4096)
            self.fc2 = nn.Dense(d)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.BN1(x)
        x = self.conv3(x)
        x = self.BN2(x)
        #x = self.pool1(x)
        x = spatial_pyramid_pool(x, 1, x.shape, self.output_num)        
        fc1 = self.fc1(self.drop(x))
        fc2 = self.fc2(fc1)
        
        return fc2

In [34]:
def project(x, y, X1, X2, X3, Y, x_test, y_test):
    
    ctx = mx.gpu()
    batch_size = 512
    random.seed(47)
    
    ArrayDataset(X1, X2, X3, Y)
    train_iter = gdata.DataLoader(gdata.ArrayDataset(X1, X2, X3, Y), batch_size, shuffle=True)    
    test_iter = gdata.DataLoader(gdata.ArrayDataset(x_test, y_test), batch_size, shuffle=True)
    x_iter = gdata.DataLoader(gdata.ArrayDataset(x, y), batch_size, shuffle=False)
    print('Data Load Success!')
    
#     net = DenseNet()
    
#     conv_arch = ((1, 64), (1, 128), (2, 256), (2, 512), (2, 512))
#     net = vgg(conv_arch)
    net = alexNet()
#     net = SPP_CNN()
#     net.hybridize()
#     net = baseNet() 
    #net.collect_params().initialize(mx.init.Uniform(scale=0.1), ctx=ctx) 
    
    net.initialize(ctx=ctx, init=init.Xavier())
    print('Build Net Success!')

    triplet_loss = gloss.TripletLoss()
    trainer_triplet = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.0001})
    
    print('---------------->')
    best_mrr = 0
    predict_time = []
    for epoch in range(10):      
        curr_loss = 0.0
        for i, (a, b, c, label) in enumerate(train_iter):            
            a = a.as_in_context(ctx)
            b = b.as_in_context(ctx)
            c = c.as_in_context(ctx)            
            
            anc_ins, pos_ins, neg_ins = a, b, c
            with autograd.record():
                inter1 = net(anc_ins)
                inter2 = net(pos_ins)
                inter3 = net(neg_ins)
                loss = triplet_loss(inter1, inter2, inter3)  # Triplet Loss
            
            loss.backward()
            trainer_triplet.step(batch_size)
            curr_loss = mx.nd.mean(loss).asscalar()
            # print('Epoch: %s, Batch: %s, Triplet Loss: %s' % (epoch, i, curr_loss))
        
        print('Epoch: %s, Triplet Loss: %s' % (epoch, curr_loss)) 
        evas = evaluate_net(net, x_iter, test_iter, ctx) #mrr,acc1,acc3,acc5,acc10
        print(evas[0], ' |', evas[1],' |', evas[2],' |', evas[3], ' |', evas[4])
        predict_time.append(evas[5])
        
        #if evas[0] > best_mrr:
            #best_mrr = evas[0]
            #net.collect_params().save('SPP+CNN.params')
            #net.save_params('SPP+CNN.params')
            #pickle.dump(net.collect_params(), open('SPP+CNN.pkl', 'wb'))
            #save(net.collect_params(), 'SPP+CNN.params')
            #net.export("net", epoch=2)
    
    predict_time = np.array(predict_time).mean()
    print('Running time: %s Seconds'%(predict_time))

In [41]:
def project_dimention(x, y, X1, X2, X3, Y, x_test, y_test, d):
    
    ctx = mx.gpu()
    batch_size = 512
    random.seed(47)
    
    ArrayDataset(X1, X2, X3, Y)
    train_iter = gdata.DataLoader(gdata.ArrayDataset(X1, X2, X3, Y), batch_size, shuffle=True)    
    test_iter = gdata.DataLoader(gdata.ArrayDataset(x_test, y_test), batch_size, shuffle=True)
    x_iter = gdata.DataLoader(gdata.ArrayDataset(x, y), batch_size, shuffle=False)
    print('Data Load Success!')
    
#     net = DenseNet()
    
#     conv_arch = ((1, 64), (1, 128), (2, 256), (2, 512), (2, 512))
#     net = vgg(conv_arch)
#     net = alexNet()
    net = SPP_CNN(d)
#     net.hybridize()
#     net = baseNet() 
    #net.collect_params().initialize(mx.init.Uniform(scale=0.1), ctx=ctx) 
    
    net.initialize(ctx=ctx, init=init.Xavier())
    print('Build Net Success!')

    triplet_loss = gloss.TripletLoss()
    trainer_triplet = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.0001})
    
    print('---------------->')
    best_mrr = 0
    predict_time = []
    for epoch in range(10):      
        curr_loss = 0.0
        for i, (a, b, c, label) in enumerate(train_iter):            
            a = a.as_in_context(ctx)
            b = b.as_in_context(ctx)
            c = c.as_in_context(ctx)            
            
            anc_ins, pos_ins, neg_ins = a, b, c
            with autograd.record():
                inter1 = net(anc_ins)
                inter2 = net(pos_ins)
                inter3 = net(neg_ins)
                loss = triplet_loss(inter1, inter2, inter3)  # Triplet Loss
            
            loss.backward()
            trainer_triplet.step(batch_size)
            curr_loss = mx.nd.mean(loss).asscalar()
            # print('Epoch: %s, Batch: %s, Triplet Loss: %s' % (epoch, i, curr_loss))
        
        print('Epoch: %s, Triplet Loss: %s' % (epoch, curr_loss)) 
        evas = evaluate_net(net, x_iter, test_iter, ctx) #mrr,acc1,acc3,acc5,acc10
        print(evas[0], ' |', evas[1],' |', evas[2],' |', evas[3], ' |', evas[4])
        predict_time.append(evas[5])
        
        #if evas[0] > best_mrr:
            #best_mrr = evas[0]
            #net.collect_params().save('SPP+CNN.params')
            #net.save_params('SPP+CNN.params')
            #pickle.dump(net.collect_params(), open('SPP+CNN.pkl', 'wb'))
            #save(net.collect_params(), 'SPP+CNN.params')
            #net.export("net", epoch=2)
    
    predict_time = np.array(predict_time).mean()
    print('Running time: %s Seconds'%(predict_time))

In [22]:
# baseNet MLP
project(x, y, X1, X2, X3, Y, x_test, y_test) #

Data Load Success!
Build Net Success!
---------------->
Epoch: 0, Triplet Loss: 0.05748183
0.33896669611434826  | 0.28  | 0.376  | 0.396  | 0.44
Epoch: 1, Triplet Loss: 0.05593499
0.3801647845234915  | 0.332  | 0.406  | 0.434  | 0.468
Epoch: 2, Triplet Loss: 0.047013484
0.3639525871118377  | 0.314  | 0.388  | 0.408  | 0.45
Epoch: 3, Triplet Loss: 0.03115404
0.3845119723465649  | 0.342  | 0.398  | 0.422  | 0.472
Epoch: 4, Triplet Loss: 0.06300057
0.369451400241989  | 0.326  | 0.392  | 0.416  | 0.444
Epoch: 5, Triplet Loss: 0.029189372
0.36257050438803495  | 0.312  | 0.388  | 0.422  | 0.45
Epoch: 6, Triplet Loss: 0.014166373
0.3555083419443531  | 0.31  | 0.374  | 0.4  | 0.438
Epoch: 7, Triplet Loss: 0.013283009
0.3655007846426245  | 0.314  | 0.396  | 0.416  | 0.466
Epoch: 8, Triplet Loss: 0.017639471
0.36980247625113  | 0.32  | 0.386  | 0.422  | 0.466
Epoch: 9, Triplet Loss: 0.015168813
0.36676623100537575  | 0.314  | 0.392  | 0.434  | 0.47
Running time: 1.482315899999992 Seconds


In [31]:
# CNN+SPP
project(x, y, X1, X2, X3, Y, x_test, y_test) #

Data Load Success!
Build Net Success!
---------------->
Epoch: 0, Triplet Loss: 1.0574257
0.5404037916147338  | 0.48  | 0.566  | 0.62  | 0.672
Epoch: 1, Triplet Loss: 12.189172
0.5551979712693433  | 0.482  | 0.596  | 0.65  | 0.696
Epoch: 2, Triplet Loss: 2.1697836
0.5968348598670644  | 0.534  | 0.622  | 0.666  | 0.716
Epoch: 3, Triplet Loss: 8.581271
0.5998539421544212  | 0.532  | 0.64  | 0.666  | 0.714
Epoch: 4, Triplet Loss: 0.62379014
0.6013133456055388  | 0.536  | 0.632  | 0.68  | 0.726
Epoch: 5, Triplet Loss: 0.4765452
0.6414196641974217  | 0.576  | 0.678  | 0.714  | 0.752
Epoch: 6, Triplet Loss: 1.9760455
0.609560500178526  | 0.546  | 0.644  | 0.692  | 0.734
Epoch: 7, Triplet Loss: 0.0
0.6358650546631764  | 0.582  | 0.658  | 0.698  | 0.75
Epoch: 8, Triplet Loss: 0.0
0.6297510589032125  | 0.566  | 0.664  | 0.694  | 0.736
Epoch: 9, Triplet Loss: 0.115688525
0.6199977645797694  | 0.568  | 0.638  | 0.668  | 0.732
Running time: 1.483275699999922 Seconds


In [27]:
# vgg
project(x, y, X1, X2, X3, Y, x_test, y_test) #

Data Load Success!
Build Net Success!
---------------->
Epoch: 0, Triplet Loss: 0.3544389
0.0171392692497552  | 0.008  | 0.01  | 0.02  | 0.03
Epoch: 1, Triplet Loss: 0.34064183
0.022094645803914018  | 0.01  | 0.024  | 0.028  | 0.03
Epoch: 2, Triplet Loss: 0.32877266
0.0283796782464776  | 0.014  | 0.028  | 0.032  | 0.044
Epoch: 3, Triplet Loss: 0.13939273
0.03585782170755398  | 0.014  | 0.028  | 0.034  | 0.06
Epoch: 4, Triplet Loss: 0.15885502
0.02946155465788756  | 0.004  | 0.018  | 0.044  | 0.07
Epoch: 5, Triplet Loss: 0.21264145
0.041243682104590056  | 0.016  | 0.032  | 0.046  | 0.078
Epoch: 6, Triplet Loss: 0.16375843
0.03453407157513352  | 0.008  | 0.02  | 0.03  | 0.072
Epoch: 7, Triplet Loss: 0.08338602
0.04237494000731938  | 0.012  | 0.036  | 0.054  | 0.078
Epoch: 8, Triplet Loss: 0.12809478
0.03714770138138281  | 0.012  | 0.024  | 0.034  | 0.064
Epoch: 9, Triplet Loss: 0.051326178
0.03479277454743044  | 0.008  | 0.022  | 0.042  | 0.068


In [57]:
# cnn without spp
project(x, y, X1, X2, X3, Y, x_test, y_test) #

Data Load Success!
Build Net Success!
---------------->
Epoch: 0, Triplet Loss: 0.8404731
0.3652207034062965  | 0.292  | 0.39  | 0.428  | 0.494
Epoch: 1, Triplet Loss: 0.09429787
0.39998121125007974  | 0.326  | 0.432  | 0.48  | 0.538
Epoch: 2, Triplet Loss: 0.2510015
0.44879599524158603  | 0.366  | 0.484  | 0.528  | 0.6
Epoch: 3, Triplet Loss: 0.13852544
0.464679765851558  | 0.39  | 0.484  | 0.554  | 0.608
Epoch: 4, Triplet Loss: 0.05552167
0.4816901232783938  | 0.404  | 0.512  | 0.564  | 0.636
Epoch: 5, Triplet Loss: 0.07329809
0.48877342203862184  | 0.408  | 0.522  | 0.59  | 0.662
Epoch: 6, Triplet Loss: 0.004454679
0.499692545946783  | 0.414  | 0.542  | 0.586  | 0.644
Epoch: 7, Triplet Loss: 0.013603657
0.49834918114022153  | 0.422  | 0.536  | 0.586  | 0.654
Epoch: 8, Triplet Loss: 0.102632135
0.49272386649286937  | 0.41  | 0.526  | 0.582  | 0.658
Epoch: 9, Triplet Loss: 0.021918485
0.5069912411090075  | 0.43  | 0.532  | 0.588  | 0.65
Running time: 1.4822194999998828 Seconds


In [41]:
# spp_cnn 64
project(x, y, X1, X2, X3, Y, x_test, y_test) #

Data Load Success!
Build Net Success!
---------------->
Epoch: 0, Triplet Loss: 0.39489725
0.3593629833199123  | 0.29  | 0.386  | 0.426  | 0.474
Epoch: 1, Triplet Loss: 0.27311426
0.40839078457002614  | 0.346  | 0.432  | 0.476  | 0.522
Epoch: 2, Triplet Loss: 0.19635671
0.45557784138156376  | 0.386  | 0.486  | 0.52  | 0.588
Epoch: 3, Triplet Loss: 0.14930607
0.45307056795664036  | 0.368  | 0.502  | 0.54  | 0.592
Epoch: 4, Triplet Loss: 0.08219996
0.48832361886895415  | 0.41  | 0.518  | 0.564  | 0.64
Epoch: 5, Triplet Loss: 0.031281315
0.49296712932806436  | 0.414  | 0.526  | 0.584  | 0.642
Epoch: 6, Triplet Loss: 0.0023213422
0.5147549570745591  | 0.444  | 0.54  | 0.584  | 0.662
Epoch: 7, Triplet Loss: 0.005821974
0.5127627968631281  | 0.432  | 0.558  | 0.6  | 0.676
Epoch: 8, Triplet Loss: 0.033547647
0.5103186555659616  | 0.43  | 0.548  | 0.598  | 0.65
Epoch: 9, Triplet Loss: 0.04915521
0.4913074882684889  | 0.42  | 0.516  | 0.566  | 0.63


In [44]:
# spp_cnn 32
project(x, y, X1, X2, X3, Y, x_test, y_test) #

Data Load Success!
Build Net Success!
---------------->
Epoch: 0, Triplet Loss: 6.312211
0.3625749910045747  | 0.286  | 0.396  | 0.428  | 0.498
Epoch: 1, Triplet Loss: 10.240993
0.4414656514087155  | 0.368  | 0.474  | 0.522  | 0.584
Epoch: 2, Triplet Loss: 1.5730058
0.4684083027438756  | 0.384  | 0.496  | 0.572  | 0.646
Epoch: 3, Triplet Loss: 4.003873
0.4663273400067392  | 0.388  | 0.496  | 0.55  | 0.62
Epoch: 4, Triplet Loss: 1.9408531
0.49299108605026143  | 0.412  | 0.53  | 0.586  | 0.648
Epoch: 5, Triplet Loss: 1.2305316
0.4988914982741626  | 0.416  | 0.538  | 0.588  | 0.654
Epoch: 6, Triplet Loss: 0.26069236
0.46948380826406383  | 0.396  | 0.506  | 0.558  | 0.6
Epoch: 7, Triplet Loss: 0.0
0.5007973660619676  | 0.436  | 0.514  | 0.572  | 0.624
Epoch: 8, Triplet Loss: 0.98329544
0.4669172380617659  | 0.392  | 0.498  | 0.55  | 0.61
Epoch: 9, Triplet Loss: 0.0
0.4496086613306121  | 0.374  | 0.482  | 0.538  | 0.588


In [37]:
# alexnet
project(x, y, X1, X2, X3, Y, x_test, y_test) #

Data Load Success!
Build Net Success!
---------------->
Epoch: 0, Triplet Loss: 0.99964106
0.13953854088447526  | 0.108  | 0.152  | 0.168  | 0.196
Epoch: 1, Triplet Loss: 0.99510497
0.1368170878312362  | 0.104  | 0.15  | 0.168  | 0.196
Epoch: 2, Triplet Loss: 0.99654603
0.1367225897497655  | 0.106  | 0.148  | 0.166  | 0.192
Epoch: 3, Triplet Loss: 0.9984805
0.1347471251857719  | 0.104  | 0.146  | 0.166  | 0.186
Epoch: 4, Triplet Loss: 0.99585825
0.13317837605710398  | 0.102  | 0.144  | 0.162  | 0.186
Epoch: 5, Triplet Loss: 0.9954663
0.13217355762466956  | 0.1  | 0.144  | 0.162  | 0.186
Epoch: 6, Triplet Loss: 1.0011123
0.1305849720695378  | 0.1  | 0.144  | 0.158  | 0.186
Epoch: 7, Triplet Loss: 0.99677783
0.12863030517929422  | 0.098  | 0.142  | 0.156  | 0.182
Epoch: 8, Triplet Loss: 0.99871874
0.12742372228190782  | 0.098  | 0.14  | 0.156  | 0.176
Epoch: 9, Triplet Loss: 0.997101
0.12670435757420587  | 0.098  | 0.136  | 0.156  | 0.174
Running time: 1.414435500000036 Seconds


In [22]:
#DenseNet
project(x, y, X1, X2, X3, Y, x_test, y_test) #

Data Load Success!
Build Net Success!
---------------->
Epoch: 0, Triplet Loss: 0.07956708
0.26748365481350755  | 0.196  | 0.298  | 0.336  | 0.404
Epoch: 1, Triplet Loss: 0.007817073
0.29395849446892625  | 0.22  | 0.33  | 0.364  | 0.42
Epoch: 2, Triplet Loss: 0.04763902
0.32711167469268077  | 0.248  | 0.366  | 0.408  | 0.464
Epoch: 3, Triplet Loss: 0.13865106
0.34204625581112896  | 0.27  | 0.37  | 0.42  | 0.48
Epoch: 4, Triplet Loss: 0.017503103
0.33373705984884194  | 0.258  | 0.368  | 0.41  | 0.466
Epoch: 5, Triplet Loss: 0.0090175355
0.36694371091444383  | 0.302  | 0.39  | 0.434  | 0.486
Epoch: 6, Triplet Loss: 0.0071848677
0.3532540148937235  | 0.274  | 0.39  | 0.432  | 0.494
Epoch: 7, Triplet Loss: 0.0
0.37417948552659164  | 0.294  | 0.41  | 0.446  | 0.506
Epoch: 8, Triplet Loss: 0.01272564
0.3718448255922297  | 0.31  | 0.4  | 0.43  | 0.47
Epoch: 9, Triplet Loss: 0.027893284
0.36826230871672044  | 0.288  | 0.406  | 0.452  | 0.5
Running time: 1.516698299999979 Seconds
